In [ ]:
import tkinter as tk
from tkinter import ttk, messagebox
import pyodbc
import pandas as pd
from azure.identity import DefaultAzureCredential
from azure.keyvault.secrets import SecretClient

# Função para buscar as credenciais do Key Vault
def retorna_credenciais(secret_name):
    KEY_VAULT_URL = "https://kv-academy-01.vault.azure.net/"
    credential = DefaultAzureCredential()
    client = SecretClient(vault_url=KEY_VAULT_URL, credential=credential)
    retrieved_secret = client.get_secret(secret_name)
    return retrieved_secret.value

# Obter credenciais seguras
server = retorna_credenciais("db-academy-server")
database = retorna_credenciais("db-academy-database")
username = retorna_credenciais("db-academy-user")
password = retorna_credenciais("db-academy-password")
driver = '{ODBC Driver 18 for SQL Server}'

# Montar string de conexão usando pyodbc
connection_string = (
    f"DRIVER={driver};"
    f"SERVER={server};"
    f"DATABASE={database};"
    f"UID={username};"
    f"PWD={password};"
)

# Criar a conexão com o banco de dados
try:
    conn = pyodbc.connect(connection_string)
    cursor = conn.cursor()
    print("Conexão com o banco de dados bem-sucedida!")
except Exception as e:
    print(f"Erro ao conectar ao banco de dados: {e}")
    raise e

In [ ]:
# Função para popular a tabela de resultados
def populate_table(df):
    for row in result_table.get_children():
        result_table.delete(row)

    if not df.empty:
        for _, row in df.iterrows():
            result_table.insert("", "end", values=list(row))
    else:
        messagebox.showinfo("Resultado", "Nenhum resultado encontrado.")

# Funções CRUD
def create_transaction():
    try:
        query = f"""
        INSERT INTO dbo.projeto_daniel_danillo_matheus 
        (Tempo, Tipo, Valor, ContaOrigem, SaldoAntigoOrigem, SaldoNovoOrigem, ContaDestino, SaldoAntigoDestino, SaldoNovoDestino, Fraude)
        VALUES ({tempo.get()}, '{tipo.get()}', {valor.get()}, '{conta_origem.get()}', {saldo_antigo_origem.get()}, {saldo_novo_origem.get()},
                '{conta_destino.get()}', {saldo_antigo_destino.get()}, {saldo_novo_destino.get()}, {fraude.get()})
        """
        cursor.execute(query)
        conn.commit()
        messagebox.showinfo("Sucesso", "Transação criada com sucesso!")
    except Exception as e:
        messagebox.showerror("Erro", str(e))

def read_transactions():
    try:
        query = "SELECT * FROM dbo.projeto_daniel_danillo_matheus"
        user_condition = condition_read.get().strip()

        if user_condition:
            query += f" WHERE {user_condition}"

        df = pd.read_sql(query, conn)
        populate_table(df)
    except Exception as e:
        messagebox.showerror("Erro", str(e))

def fetch_last_10_records():
    try:
        query = "SELECT TOP 10 * FROM dbo.projeto_daniel_danillo_matheus ORDER BY Tempo DESC"
        df = pd.read_sql(query, conn)
        populate_table(df)
    except Exception as e:
        messagebox.showerror("Erro", str(e))

def update_transaction():
    try:
        update_str = f"{coluna.get()} = '{novo_valor.get()}'"
        query = f"UPDATE dbo.projeto_daniel_danillo_matheus SET {update_str} WHERE {condition_update.get()}"
        cursor.execute(query)
        conn.commit()
        messagebox.showinfo("Sucesso", "Transação atualizada com sucesso!")
    except Exception as e:
        messagebox.showerror("Erro", str(e))

def delete_transaction():
    try:
        query = f"DELETE FROM dbo.projeto_daniel_danillo_matheus WHERE {condition_delete.get()}"
        cursor.execute(query)
        conn.commit()
        messagebox.showinfo("Sucesso", "Transação excluída com sucesso!")
    except Exception as e:
        messagebox.showerror("Erro", str(e))

# Função de tutorial com exemplos
def show_tutorial():
    tutorial_text = (
        "Tutorial do CRUD\n\n"
        "1. Aba Create: Preencha os campos e clique em 'Criar Transação' para adicionar uma nova transação ao banco de dados.\n"
        "   Exemplo: Para adicionar uma transação, preencha os campos com os seguintes dados:\n"
        "   - Tempo: 2024-11-29 14:30:00\n"
        "   - Tipo: Transferência\n"
        "   - Valor: 1000.50\n"
        "   - Conta Origem: 12345\n"
        "   - Saldo Antigo Origem: 5000.00\n"
        "   - Saldo Novo Origem: 4000.00\n"
        "   - Conta Destino: 67890\n"
        "   - Saldo Antigo Destino: 1000.00\n"
        "   - Saldo Novo Destino: 2000.50\n"
        "   - Fraude: Não\n"
        "   Clique em 'Criar Transação' para adicionar essa transação ao banco de dados.\n\n"
        
        "2. Aba Read: Preencha a condição (opcional) para filtrar e clique em 'Ler Transações'.\n"
        "   Exemplo: Se você quiser visualizar todas as transações realizadas por uma conta específica, insira a condição:\n"
        "   - Condição: ContaOrigem = '12345'\n"
        "   Isso exibirá todas as transações em que a ContaOrigem seja 12345.\n\n"
        
        "3. Aba Update: Selecione a coluna a ser atualizada, insira o novo valor e a condição para aplicar a alteração.\n"
        "   Exemplo: Para atualizar o saldo de uma conta, preencha os campos com os seguintes dados:\n"
        "   - Coluna: SaldoNovoOrigem\n"
        "   - Novo Valor: 4500.00\n"
        "   - Condição: ContaOrigem = '12345' AND Tipo = 'Transferência'\n"
        "   Isso atualizará o saldo da ContaOrigem 12345 para 4500.00 para todas as transações do tipo 'Transferência'.\n\n"
        
        "4. Aba Delete: Insira a condição para deletar uma transação.\n"
        "   Exemplo: Se você quiser deletar uma transação específica, insira a condição:\n"
        "   - Condição: ContaOrigem = '12345' AND Tipo = 'Transferência'\n"
        "   Isso deletará todas as transações de transferência realizadas pela ContaOrigem 12345.\n\n"
        
        "5. Exibir resultados: Os resultados das transações serão exibidos na tabela abaixo.\n\n"
        "Utilize os botões para realizar as ações desejadas no banco de dados."
    )
    messagebox.showinfo("Tutorial", tutorial_text)

# Função para finalizar a aplicação
def close_application():
    try:
        conn.close()  # Fecha a conexão com o banco de dados
    except Exception as e:
        print(f"Erro ao fechar a conexão: {e}")
    root.destroy()  # Encerra o loop principal do Tkinter de forma segura

# Configuração da interface gráfica
root = tk.Tk()
root.title("CRUD - Tkinter")
root.geometry("1200x600")

notebook = ttk.Notebook(root)
notebook.pack(fill=tk.BOTH, expand=True)

create_frame = ttk.Frame(notebook)
read_frame = ttk.Frame(notebook)
update_frame = ttk.Frame(notebook)
delete_frame = ttk.Frame(notebook)

notebook.add(create_frame, text="Create")
notebook.add(read_frame, text="Read")
notebook.add(update_frame, text="Update")
notebook.add(delete_frame, text="Delete")

# Aba Create
tk.Label(create_frame, text="Tempo").grid(row=0, column=0, sticky="w", padx=10, pady=5)
tempo = tk.Entry(create_frame, width=25)
tempo.grid(row=0, column=1, padx=10, pady=5)

tk.Label(create_frame, text="Tipo").grid(row=1, column=0, sticky="w", padx=10, pady=5)
tipo = tk.Entry(create_frame, width=25)
tipo.grid(row=1, column=1, padx=10, pady=5)

tk.Label(create_frame, text="Valor").grid(row=2, column=0, sticky="w", padx=10, pady=5)
valor = tk.Entry(create_frame, width=25)
valor.grid(row=2, column=1, padx=10, pady=5)

tk.Label(create_frame, text="Conta Origem").grid(row=3, column=0, sticky="w", padx=10, pady=5)
conta_origem = tk.Entry(create_frame, width=25)
conta_origem.grid(row=3, column=1, padx=10, pady=5)

tk.Label(create_frame, text="Saldo Antigo Origem").grid(row=4, column=0, sticky="w", padx=10, pady=5)
saldo_antigo_origem = tk.Entry(create_frame, width=25)
saldo_antigo_origem.grid(row=4, column=1, padx=10, pady=5)

tk.Label(create_frame, text="Saldo Novo Origem").grid(row=5, column=0, sticky="w", padx=10, pady=5)
saldo_novo_origem = tk.Entry(create_frame, width=25)
saldo_novo_origem.grid(row=5, column=1, padx=10, pady=5)

tk.Label(create_frame, text="Conta Destino").grid(row=6, column=0, sticky="w", padx=10, pady=5)
conta_destino = tk.Entry(create_frame, width=25)
conta_destino.grid(row=6, column=1, padx=10, pady=5)

tk.Label(create_frame, text="Saldo Antigo Destino").grid(row=7, column=0, sticky="w", padx=10, pady=5)
saldo_antigo_destino = tk.Entry(create_frame, width=25)
saldo_antigo_destino.grid(row=7, column=1, padx=10, pady=5)

tk.Label(create_frame, text="Saldo Novo Destino").grid(row=8, column=0, sticky="w", padx=10, pady=5)
saldo_novo_destino = tk.Entry(create_frame, width=25)
saldo_novo_destino.grid(row=8, column=1, padx=10, pady=5)

tk.Label(create_frame, text="Fraude").grid(row=9, column=0, sticky="w", padx=10, pady=5)
fraude = tk.Entry(create_frame, width=25)
fraude.grid(row=9, column=1, padx=10, pady=5)

# Botões para a aba Create
tk.Button(create_frame, text="Criar Transação", command=create_transaction).grid(row=10, column=0, columnspan=2, pady=10)

# Aba Read
tk.Label(read_frame, text="Condição (opcional)").grid(row=0, column=0, padx=10, pady=5)
condition_read = tk.Entry(read_frame, width=50)
condition_read.grid(row=0, column=1, padx=10, pady=5)

# Botões para a aba Read
tk.Button(read_frame, text="Ler Transações", command=read_transactions).grid(row=1, column=0, columnspan=2, pady=10)
tk.Button(read_frame, text="Últimas 10", command=fetch_last_10_records).grid(row=2, column=0, columnspan=2, pady=10)

# Tabela para resultados
columns = ("Tempo", "Tipo", "Valor", "ContaOrigem", "SaldoAntigoOrigem", "SaldoNovoOrigem", "ContaDestino", "SaldoAntigoDestino", "SaldoNovoDestino", "Fraude")
result_table = ttk.Treeview(read_frame, show="headings", height=10, columns=columns)
for col in columns:
    result_table.heading(col, text=col)
    result_table.column(col, width=120)
result_table.grid(row=3, column=0, columnspan=2, pady=10, padx=10)

# Aba Update
tk.Label(update_frame, text="Coluna").grid(row=0, column=0, padx=10, pady=5)
coluna = tk.Entry(update_frame, width=25)
coluna.grid(row=0, column=1, padx=10, pady=5)

tk.Label(update_frame, text="Novo Valor").grid(row=1, column=0, padx=10, pady=5)
novo_valor = tk.Entry(update_frame, width=25)
novo_valor.grid(row=1, column=1, padx=10, pady=5)

tk.Label(update_frame, text="Condição").grid(row=2, column=0, padx=10, pady=5)
condition_update = tk.Entry(update_frame, width=50)
condition_update.grid(row=2, column=1, padx=10, pady=5)

# Botões para a aba Update
tk.Button(update_frame, text="Atualizar Transação", command=update_transaction).grid(row=3, column=0, columnspan=2, pady=10)

# Aba Delete
tk.Label(delete_frame, text="Condição").grid(row=0, column=0, padx=10, pady=5)
condition_delete = tk.Entry(delete_frame, width=50)
condition_delete.grid(row=0, column=1, padx=10, pady=5)

# Botões para a aba Delete
tk.Button(delete_frame, text="Excluir Transação", command=delete_transaction).grid(row=1, column=0, columnspan=2, pady=10)

# Botão de tutorial
tk.Button(root, text="Tutorial", command=show_tutorial).pack(side=tk.BOTTOM, pady=10)

# Fechar a aplicação
root.protocol("WM_DELETE_WINDOW", close_application)

root.mainloop()

Conexão com o banco de dados bem-sucedida!
